# Amazon Bedrock AgentCore Runtime での Strands Agents を使用したストリーミングレスポンス

## 概要

このチュートリアルでは、Amazon Bedrock AgentCore Runtime を使用してストリーミングレスポンスを実装する方法を学びます。この例では、部分的な結果が利用可能になった時点でストリーミングする方法を示し、大量のコンテンツを生成したり、処理時間が長くかかる操作に対してより応答性の高いユーザーエクスペリエンスを提供します。


### チュートリアル詳細

|情報| 詳細|
|:--------------------|:---------------------------------------------------------------------------------|
| チュートリアルタイプ | ストリーミング付き会話形式|
| エージェントタイプ   | シングル         |
| エージェントフレームワーク | Strands Agents |
| LLM モデル          | Anthropic Claude Haiku 4.5 |
| チュートリアル構成   | AgentCore Runtime、Strands Agent、Amazon Bedrock モデルを使用したストリーミングレスポンス |
| チュートリアル分野   | クロスバーティカル                                                                   |
| 難易度              | 簡単                                                                             |
| 使用 SDK            | Amazon BedrockAgentCore Python SDK と boto3|

### チュートリアルアーキテクチャ

このチュートリアルでは、ストリーミングエージェントを AgentCore Runtime にデプロイする方法を説明します。

デモンストレーション目的で、ストリーミング機能を備えた Amazon Bedrock モデルを使用する Strands Agent を使用します。

この例では、`get_weather` と `get_time` の2つのツールを持つシンプルなエージェントを使用しますが、ストリーミングレスポンス機能を備えています。

    
<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="60%"/>
</div>

### チュートリアルの主な機能

* Amazon Bedrock AgentCore Runtime でのエージェントからのストリーミングレスポンス
* リアルタイムの部分結果配信
* ストリーミング対応の Amazon Bedrock モデルの使用
* 非同期ストリーミングをサポートする Strands Agents の使用

## 前提条件

このチュートリアルを実行するには、以下が必要です：
* Python 3.10+
* AWS 認証情報
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker が実行中

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## AgentCore Runtime へのデプロイ用にストリーミングエージェントを準備

それでは、ストリーミングエージェントを AgentCore Runtime にデプロイしましょう。ストリーミング機能は、エントリーポイント関数で async ジェネレーターまたは yield ステートメントを使用すると、AgentCore SDK によって自動的に処理されます。

ストリーミング実装の主なポイント：
* エントリーポイント関数に `async def` を使用
* チャンクが利用可能になったら `yield` を使用してストリーミング
* AgentCore SDK は Server-Sent Events（SSE）形式を自動的に処理
* クライアントは Content-Type: text/event-stream レスポンスを受信

### Amazon Bedrock モデルとストリーミングを使用した Strands Agents
Amazon Bedrock モデルを使用する Strands Agent のストリーミング実装を見てみましょう。

In [ ]:
%%writefile strands_claude_streaming.py
from strands import Agent, tool
from strands_tools import calculator # calculator ツールをインポート
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel
import asyncio
from datetime import datetime

app = BedrockAgentCoreApp()

# カスタムツールを作成 
@tool
def weather():
    """ 天気を取得 """ # ダミー実装
    return "sunny"

@tool
def get_time():
    """ 現在時刻を取得 """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[
        calculator, weather, get_time
    ],
    system_prompt="""You're a helpful assistant. You can do simple math calculations, 
    tell the weather, and provide the current time."""
)

@app.entrypoint
async def strands_agent_bedrock_streaming(payload):
    """
    ストリーミング機能付きでエージェントを呼び出す
    この関数は、非同期ジェネレーターを使用して AgentCore Runtime で
    ストリーミングレスポンスを実装する方法を示します
    """
    user_input = payload.get("prompt")
    print("ユーザー入力:", user_input)
    
    try:
        # 各チャンクを利用可能になり次第ストリーム
        async for event in agent.stream_async(user_input):
            if "data" in event:
                yield event["data"]
            
    except Exception as e:
        # ストリーミングコンテキストでエラーを適切に処理
        error_response = {"error": str(e), "type": "stream_error"}
        print(f"ストリーミングエラー: {error_response}")
        yield error_response

if __name__ == "__main__":
    app.run()

## AgentCore Runtime でのストリーミングを理解する

AgentCore Runtime でストリーミングを使用すると、いくつかのことが自動的に行われます：

### Server-Sent Events（SSE）形式
* AgentCore SDK は yield されたデータを自動的に SSE 形式に変換
* 各 yield は SSE ストリーム内の `data: ` イベントになる
* Content-Type は自動的に `text/event-stream` に設定される

### クライアント処理
* クライアントはエージェントがリクエストを処理する際にリアルタイム更新を受信
* これにより、プログレッシブなレスポンス表示とより良いユーザーエクスペリエンスが可能
* クライアントは完全なレスポンスが準備される前に部分的な結果を処理可能

### エラー処理
* ストリーミングレスポンスには適切なエラー処理を含めるべき
* エラーはストリームの一部として yield 可能
* 関数が完了するか、未処理の例外が発生するとストリームは終了

## ストリーミングエージェントを AgentCore Runtime にデプロイ

`CreateAgentRuntime` 操作は包括的な設定オプションをサポートしており、コンテナイメージ、環境変数、暗号化設定を指定できます。また、クライアントがエージェントとどのように通信するかを制御するために、プロトコル設定（HTTP、MCP）と認可メカニズムを設定することもできます。

**注意：** 運用のベストプラクティスは、CI/CD パイプラインと IaC を使用してコードをコンテナとしてパッケージ化し、ECR にプッシュすることです

このチュートリアルでは、Amazon Bedrock AgentCode Python SDK を使用して、アーティファクトを簡単にパッケージ化し、AgentCore Runtime にデプロイします。

### AgentCore Runtime デプロイメントの設定

次に、スターターツールキットを使用して、エントリーポイント、先ほど作成した実行ロール、requirements ファイルで AgentCore Runtime デプロイメントを設定します。また、起動時に Amazon ECR リポジトリを自動作成するようにスターターキットを設定します。

設定ステップ中、アプリケーションコードに基づいて Docker ファイルが生成されます

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_claude_streaming.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_claude_streaming"
)
response

### ストリーミングエージェントを AgentCore Runtime に起動

Docker ファイルができたので、ストリーミングエージェントを AgentCore Runtime に起動しましょう。これにより、Amazon ECR リポジトリと AgentCore Runtime が作成されます

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()

### AgentCore Runtime のステータス確認
AgentCore Runtime をデプロイしたので、デプロイメントステータスを確認しましょう

In [ ]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### ストリーミングで AgentCore Runtime を呼び出す

最後に、ペイロードを使用して AgentCore Runtime を呼び出し、ストリーミングレスポンスを受信できます

<div style="text-align:left">
    <img src="images/invoke.png" width="85%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({
    "prompt": 
    "what the weather is like?"
})
invoke_response

### boto3 を使用したストリーミングで AgentCore Runtime を呼び出す

AgentCore Runtime が作成されたので、任意の AWS SDK を使用して呼び出すことができます。ストリーミングレスポンスの場合、Server-Sent Events 形式を処理する必要があります。

In [ ]:
import boto3
import json
from IPython.display import Markdown, display

agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

# ストリーミングレスポンスの場合、EventStream を処理する必要がある
boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "How much is 2+1"})
)

# レスポンスがストリーミングかどうかを確認
if "text/event-stream" in boto3_response.get("contentType", ""):
    print("boto3 でストリーミングレスポンスを処理中:")
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                data = line[6:].replace('"', '')  # "data: " プレフィックスを削除
                print(f"ストリーミングチャンクを受信: {data}")
                content.append(data.replace('"', ''))
    
    # 完全なストリーミングレスポンスを表示
    full_response = " ".join(content)
    display(Markdown(full_response))
else:
    # 非ストリーミングレスポンスを処理
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"EventStream 読み取りエラー: {e}"]
    
    if events:
        try:
            response_data = json.loads(events[0].decode("utf-8"))
            display(Markdown(response_data))
        except:
            print(f"生のレスポンス: {events[0]}")

## ストリーミングレスポンスの利点

ストリーミングレスポンスにはいくつかの主要な利点があります：

### ユーザーエクスペリエンス
* **即時フィードバック**: ユーザーは利用可能になった時点で部分的な結果を確認
* **体感パフォーマンス**: 合計時間が同じでもレスポンスが速く感じられる
* **プログレッシブ表示**: 長いレスポンスを段階的に表示可能

### 技術的メリット
* **メモリ効率**: すべてをメモリにロードせずに大きなレスポンスを処理
* **タイムアウト防止**: 長時間実行される操作でのタイムアウトを回避
* **リアルタイム処理**: 利用可能になった時点でリアルタイムデータを処理

### ユースケース
* **コンテンツ生成**: 長文の執筆、レポート、ドキュメント
* **データ分析**: 複雑な計算からのプログレッシブな結果
* **マルチステップワークフロー**: 複雑なエージェント推論の進行状況を表示
* **リアルタイムモニタリング**: モニタリングエージェントからのライブ更新

## クリーンアップ（オプション）

作成した AgentCore Runtime をクリーンアップしましょう

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# おめでとうございます！

Amazon Bedrock AgentCore Runtime を使用したストリーミングエージェントの実装とデプロイに成功しました！

## 学んだこと：
* async ジェネレーターを使用したストリーミングレスポンスの実装方法
* AgentCore Runtime が SSE 形式を自動的に処理する方法
* クライアント側でストリーミングレスポンスを処理する方法
* ユーザーエクスペリエンスとパフォーマンスにおけるストリーミングの利点

## 次のステップ：
* ユースケースに応じた異なるストリーミングパターンを試す
* 複雑なマルチステップワークフロー用のカスタムストリーミングロジックを実装
* ストリーミングと Memory や Gateway などの他の AgentCore 機能の組み合わせを探る
* より良い UX のためにクライアント側のストリーミング可視化の実装を検討